Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [9]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
#print out data sample

In [2]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [4]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [6]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    9.1s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [7]:
grid_search.best_score_

0.8879844961240311

In [8]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

### Assignment Begins HERE!
## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [10]:
import pandas as pd

train = pd.read_csv('./train.csv/train.csv')
test = pd.read_csv('./test.csv')

In [4]:
train.head()

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [11]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [37]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [45]:
parameters = {
    'vect__max_df': ( 0.85, 1.0),
    'vect__max_features': (500,6000, 25),
    'clf__n_estimators':(2, 200, 2),
    'clf__max_depth':(2,32, 2)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   15.6s
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:   22.2s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

## Challenge

You're trying to achieve 75% Accuracy on your model.

In [47]:
grid_search.best_score_

0.7357497912632837

In [52]:
grid_search.predict(test)

array([1, 1], dtype=int64)

In [54]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [55]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [60]:
# Make Sure the Category is an Integer
submission.ratingCategory.value_counts()

1    935
0     87
Name: ratingCategory, dtype: int64

In [61]:
subNumber = 0

In [63]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [66]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

In [87]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators': range(2, 200, 25),
    'clf__max_depth': range(2,32, 2)
}

In [88]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [89]:
print(pipe)

Pipeline(memory=None,
         steps=[('lsi',
                 Pipeline(memory=None,
                          steps=[('vect',
                                  TfidfVectorizer(analyzer='word', binary=False,
                                                  decode_error='strict',
                                                  dtype=<class 'numpy.float64'>,
                                                  encoding='utf-8',
                                                  input='content',
                                                  lowercase=True, max_df=1.0,
                                                  max_features=None, min_df=1,
                                                  ngram_range=(1, 2), norm='l2',
                                                  preprocessor=None,
                                                  smooth_idf=True,
                                                  stop_words='english',
                                                  strip_a

In [90]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 3 folds for each of 1080 candidates, totalling 3240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   36.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed: 17.2min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed: 51.2min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed: 73.4min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 100.0min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 130.8min
[Parallel(n_jobs=-1)]: Done 3240 out of 3240 | elapsed: 133.8min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [92]:
grid_search.best_score_

0.7240048782432291

In [93]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [94]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [95]:
# Make Sure the Category is an Integer
submission.ratingCategory.value_counts()

1    969
0     53
Name: ratingCategory, dtype: int64

In [96]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [12]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [7]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [12]:
X = get_word_vectors(train['description'])

len(X) == len(train.description)

True

In [15]:
X_test = get_word_vectors(test['description'])

In [107]:
pip install xgboost


  Using cached xgboost-1.2.0-py3-none-win_amd64.whl (86.5 MB)


In [13]:
from xgboost.sklearn import XGBClassifier
XGBC = XGBClassifier()
rfc = RandomForestClassifier()

In [19]:
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
rfc.score(X, train['ratingCategory'])

1.0

In [23]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [24]:
test['ratingCategory'] = rfc.predict(X_test)

In [25]:
test[['id', 'ratingCategory']].to_csv('testSolutionSubmission.csv', header=True, index=False)

## Follow Along

In [6]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [14]:
from xgboost.sklearn import XGBClassifier
XGBC = XGBClassifier()

In [9]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', XGBC)
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [10]:
parameters = {
    'vect__max_df': ( 0.85, 1.0),
    'vect__max_features': range(500,1000, 250),
    'clf__n_estimators':(2, 200, 2),
    'clf__max_depth': range(2,32, 2),
    'clf__learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   52.1s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed: 15.1min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed: 28.1min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 36.5min
[Parallel(n_jobs=-1)]: Done 2700 out of 2700 | elapsed: 43.1min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

## Challenge

You're trying to achieve 75% Accuracy on your model.

In [11]:
grid_search.best_score_

0.7401555574841465

In [2]:
grid_search.predict(test)

NameError: name 'grid_search' is not defined

In [12]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [13]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [14]:
# Make Sure the Category is an Integer
submission.ratingCategory.value_counts()

1    887
0    135
Name: ratingCategory, dtype: int64

In [16]:
subNumber=2

In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [15]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100,  
                   algorithm='randomized',
                   n_iter=10)

In [33]:
params = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators': range(1, 160, 20),
    'clf__max_depth': range(2,32, 4),
    'clf__learning_rate': [0.1, 0.01, 0.05]
}

In [34]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', XGBC)
                ])


In [35]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', XGBC)])

In [36]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.ratingCategory)

Fitting 2 folds for each of 1728 candidates, totalling 3456 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed: 23.9min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 1768 tasks      | elapsed: 56.2min
[Parallel(n_jobs=-1)]: Done 2418 tasks      | elapsed: 80.0min
[Parallel(n_jobs=-1)]: Done 3168 tasks      | elapsed: 104.9min
[Parallel(n_jobs=-1)]: Done 3456 out of 3456 | elapsed: 115.5min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

In [37]:
grid_search.best_score_

0.7325689936425559

In [38]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [39]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [40]:
# Make Sure the Category is an Integer
submission.ratingCategory.value_counts()

1    926
0     96
Name: ratingCategory, dtype: int64

In [41]:
subNumber=3

In [42]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

# Review

To review this module: 
* Continue working on the Kaggle competition
* Find another text classification task to work on